In [1]:
from datasets import load_dataset
import pandas as pd
import emoji
import re
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from tqdm.notebook import tqdm

from transformers import pipeline
import ast
from sklearn.model_selection import train_test_split
from sklearn import metrics, model_selection
from nltk.corpus import stopwords

import gensim.models as gsm

In [2]:
# 훈련 데이터 셋 불러오기
dataset = load_dataset('go_emotions', 'simplified')
dataset = dataset['train'].to_pandas()

Using the latest cached version of the dataset since go_emotions couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'simplified' at C:\Users\김성연\.cache\huggingface\datasets\go_emotions\simplified\0.0.0\add492243ff905527e67aeb8b80c082af02207c3 (last modified on Thu Jan 11 19:31:53 2024).


In [3]:
dataset

,text,labels,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,[18],edsb738
43406,Always thought that was funny but is it a refe...,[6],ee7fdou
43407,What are you talking about? Anything bad that ...,[3],efgbhks
43408,"More like a baptism, with sexy results!",[13],ed1naf8


In [4]:
print("원래 데이터 개수: ", len(dataset))

cols = ['text', 'labels']
mid_dataset = pd.DataFrame(columns=cols)
new_dataset = pd.DataFrame(columns=cols)

# k = 0
# for i in range(len(dataset)):
#     if len(dataset.iloc[i]['labels']) == 1:
#         mid_dataset.loc[k] = dataset.iloc[i][['text', 'labels']]
#         k += 1
# print("label 1개인 데이터 추출: ", len(mid_dataset))

j=0
l=1000
for i in range(len(dataset)):
    if emoji.emoji_count(dataset.iloc[i]['text']) == 0:  # 이모지 들어있지 않은 텍스트
        if j < 1000:
            new_dataset.loc[j] = dataset.iloc[i][['text', 'labels']]
            j += 1
    else:  # 이모지 들어있는 텍스트
        if l < 5000:
            new_dataset.loc[l] = dataset.iloc[i][['text', 'labels']]
            l += 1
        
print("추출한 데이터 총 개수: ", len(new_dataset))
print("emoji 포함된 데이터 개수: ", len([text for text in new_dataset['text'] if emoji.emoji_count(text) > 0]))
print("emoji 포함되지 않은 데이터 개수: ", len([text for text in new_dataset['text'] if emoji.emoji_count(text) == 0]))

원래 데이터 개수:  43410
emoji 1개 이상 포함되어 있는 데이터 추출:  1841
emoji 포함된 데이터 개수:  841
emoji 포함되지 않은 데이터 개수:  1000


In [5]:
new_dataset

,text,labels
0,My favourite food is anything I didn't have to...,[27]
1,"Now if he does off himself, everyone will thin...",[27]
2,WHY THE FUCK IS BAYLESS ISOING,[2]
3,To make her feel threatened,[14]
4,Dirty Southern Wankers,[3]
...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]"
1837,Talked with my parents. It was so relaxing and...,[15]
1838,"OMG, is this the one that wore white? 😳","[7, 26]"
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]"


In [6]:
# # 이모지 포함되지 않은 텍스트
# print("0번 text: ", new_dataset.iloc[0]['text'], ', labels: ', new_dataset.iloc[0]['labels'])
# print("499번 text: ", new_dataset.iloc[499]['text'], ', labels: ', new_dataset.iloc[499]['labels'])

# # 이모지 포함된 텍스트
# print("500번 text: ", new_dataset.iloc[500]['text'], ', labels: ', new_dataset.iloc[500]['labels'])
# print("999번 text: ", new_dataset.iloc[999]['text'], ', labels: ', new_dataset.iloc[999]['labels'])

In [ ]:
# 불용어 리스트

stop_words = set(stopwords.words('english'))

In [7]:
# data preprocess function

def preprocess_data(text):
    emoji_list = emoji.distinct_emoji_list(text)
    emoji_string = ''.join(emoji_list)
    text = re.sub(r'(\[.+\])', '', text)
    text = re.sub(r'[^a-zA-Z\s'+emoji_string+']', ' ', text)
    for char in text:
        if char in emoji_string:
            text = re.sub(r'['+char+']', ' '+char+' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.split(' ') 
    text = [word.lower() for word in text if len(word) > 0]
    text = [word for word in text if word not in stop_words]
    return text

In [8]:
# preprocess_data function example

text = new_dataset.iloc[1838]['text']
tokenized = preprocess_data(text)
tokenized

['OMG', 'is', 'this', 'the', 'one', 'that', 'wore', 'white', '😳']

In [9]:
# new_dataset preprocess

new_dataset['tokenized_text'] = new_dataset['text'].apply(preprocess_data)
new_dataset.iloc[499:504][['text', 'tokenized_text']]

,text,tokenized_text
487,"[NAME], [NAME] is the most underrated [NAME]. ...","[I, love, that, guy]"
488,I was actually a little lost here and there fo...,"[I, was, actually, a, little, lost, here, and,..."
489,Oh my god. Don't you know it's illegal to shar...,"[Oh, my, god, Don, t, you, know, it, s, illega..."
490,[NAME] and [NAME] should be there too E: Damn ...,"[should, be, there, too, E, Damn, didn, t, thi..."
491,I think [NAME] could've been a long term quali...,"[I, think, like, a, bad, habit]"


In [10]:
new_dataset

,text,labels,tokenized_text
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t..."
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]"
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]"
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]"
...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ..."
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax..."
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]"
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We..."


In [11]:
# word emotion analysis

classifier = pipeline(task="text-classification",
                      model="SamLowe/roberta-base-go_emotions",
                      top_k=None)

In [12]:
# emotion classes (medium output)

idx_to_emotion = {
    0: 'admiration',
    1: 'amusement',
    2: 'anger',
    3: 'annoyance',
    4: 'approval',
    5: 'caring',
    6: 'confusion',
    7: 'curiosity',
    8: 'desire',
    9: 'disappointment',
    10: 'disapproval',
    11: 'disgust',
    12: 'embarrassment',
    13: 'excitement',
    14: 'fear',
    15: 'gratitude',
    16: 'grief',
    17: 'joy',
    18: 'love',
    19: 'nervousness',
    20: 'optimism',
    21: 'pride',
    22: 'realization',
    23: 'relief',
    24: 'remorse',
    25: 'sadness',
    26: 'surprise',
    27: 'neutral'
}

emotion_to_idx = {
    'admiration': 0,
    'amusement': 1,
    'anger': 2,
    'annoyance': 3,
    'approval': 4,
    'caring': 5,
    'confusion': 6,
    'curiosity': 7,
    'desire': 8,
    'disappointment': 9,
    'disapproval': 10,
    'disgust': 11,
    'embarrassment': 12,
    'excitement': 13,
    'fear': 14,
    'gratitude': 15,
    'grief': 16,
    'joy': 17,
    'love': 18,
    'nervousness': 19,
    'optimism': 20,
    'pride': 21,
    'realization': 22,
    'relief': 23,
    'remorse': 24,
    'sadness': 25,
    'surprise': 26,
    'neutral': 27
}

In [13]:
print("the number of emotion classes: ", len((idx_to_emotion)))
print("idx_to_emotion: ", idx_to_emotion)
print("emotion_to_idx: ", emotion_to_idx)

the number of emotion classes:  28
idx_to_emotion:  {0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}
emotion_to_idx:  {'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'caring': 5, 'confusion': 6, 'curiosity': 7, 'desire': 8, 'disappointment': 9, 'disapproval': 10, 'disgust': 11, 'embarrassment': 12, 'excitement': 13, 'fear': 14, 'gratitude': 15, 'grief': 16, 'joy': 17, 'love': 18, 'nervousness': 19, 'optimism': 20, 'pride': 21, 'realization': 22, 'relief': 23, 'remorse': 24, 'sadness': 25, 'surprise': 26, 'neutral': 27}


In [14]:
def word_emotion(text):
    emotions = classifier(text)
    output = [0]*28
    for emotion in emotions[0]:
        num_emotion = emotion_to_idx[emotion['label']]
        output[num_emotion] = emotion['score']
    return output

In [15]:
def add_emotion_col(data):
    tokenized_text = data['tokenized_text']
    col_emotion = []
    for text in tokenized_text:
        text_emotion = []
        for i in range(len(text)):
            if emoji.emoji_count(text[i]) > 0:
                emoji_name = emoji.demojize(text[i])[1:-1]
                emoji_emotion = word_emotion(emoji_name)
                text_emotion.append(emoji_emotion)
            else:
                text_emotion.append(word_emotion(text[i]))
        col_emotion.append(text_emotion)
    data['emotion'] = col_emotion
    return data

In [16]:
new_dataset = add_emotion_col(new_dataset)

In [17]:
new_dataset

,text,labels,tokenized_text,emotion
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t...","[[0.003761823521926999, 0.0014409106224775314,..."
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi...","[[0.0026063367258757353, 0.0013388615334406495..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]","[[0.0019205614225938916, 0.001291286083869636,..."
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]","[[0.0035849562846124172, 0.001992727629840374,..."
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]","[[0.0049132914282381535, 0.005827016197144985,..."
...,...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ...","[[0.0035167797468602657, 0.0014414149336516857..."
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax...","[[0.002665032632648945, 0.002374983159825206, ..."
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]","[[0.02350100688636303, 0.008980895392596722, 0..."
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We...","[[0.002730799373239279, 0.0026100073009729385,..."


In [18]:
dataset = new_dataset.copy()

In [19]:
# # loading new_dataset

# new_dataset = pd.read_csv('new_dataset.csv', encoding='utf-8')
# new_dataset

In [20]:
# def process_emotion_col(row):
#     try:
#         py_list = ast.literal_eval(row['emotion'])
#         return py_list
#     except Exception as e:
#         print("에러가 발생했습니다")
#         return None

In [21]:
# # 문자열 리스트로 변환
# new_dataset['tokenized_text'] = new_dataset['tokenized_text'].apply(ast.literal_eval)
# new_dataset['labels'] = new_dataset['labels'].apply(ast.literal_eval)
# new_dataset['emotion'] = new_dataset.apply(process_emotion_col, axis=1)
# new_dataset = new_dataset.dropna()

In [22]:
dataset

,text,labels,tokenized_text,emotion
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t...","[[0.003761823521926999, 0.0014409106224775314,..."
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi...","[[0.0026063367258757353, 0.0013388615334406495..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]","[[0.0019205614225938916, 0.001291286083869636,..."
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]","[[0.0035849562846124172, 0.001992727629840374,..."
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]","[[0.0049132914282381535, 0.005827016197144985,..."
...,...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ...","[[0.0035167797468602657, 0.0014414149336516857..."
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax...","[[0.002665032632648945, 0.002374983159825206, ..."
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]","[[0.02350100688636303, 0.008980895392596722, 0..."
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We...","[[0.002730799373239279, 0.0026100073009729385,..."


In [23]:
#padding하기 위해

print("데이터 중 최대 단어 수: ", max([len(instance) for instance in dataset['emotion']]))

데이터 중 최대 단어 수:  47


In [24]:
# padding

def flattening_padding(col):
    tensor = [torch.tensor(instance).reshape(-1) for instance in col]
    padded_tensor = pad_sequence(tensor, batch_first=True, padding_value=0)
    return [item for item in padded_tensor]

In [25]:
dataset['flattened_padded_emotion'] = flattening_padding(dataset['emotion'])

In [26]:
print("인스턴스 별 emotion_embedding 벡터 길이: ", len(dataset['flattened_padded_emotion'][0]))

인스턴스 별 emotion_embedding 벡터 길이:  1316


In [27]:
#tesor들 전부 일반 리스트로 & flattening
result = [instance.tolist() for instance in dataset['flattened_padded_emotion']]
dataset['flattened_padded_emotion'] = result

In [28]:
dataset

,text,labels,tokenized_text,emotion,flattened_padded_emotion
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t...","[[0.003761823521926999, 0.0014409106224775314,...","[0.003761823521926999, 0.0014409106224775314, ..."
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi...","[[0.0026063367258757353, 0.0013388615334406495...","[0.0026063367258757353, 0.0013388615334406495,..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]","[[0.0019205614225938916, 0.001291286083869636,...","[0.0019205614225938916, 0.001291286083869636, ..."
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]","[[0.0035849562846124172, 0.001992727629840374,...","[0.0035849562846124172, 0.001992727629840374, ..."
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]","[[0.0049132914282381535, 0.005827016197144985,...","[0.0049132914282381535, 0.005827016197144985, ..."
...,...,...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ...","[[0.0035167797468602657, 0.0014414149336516857...","[0.0035167797468602657, 0.0014414149336516857,..."
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax...","[[0.002665032632648945, 0.002374983159825206, ...","[0.002665032632648945, 0.002374983159825206, 0..."
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]","[[0.02350100688636303, 0.008980895392596722, 0...","[0.02350100688636303, 0.008980895392596722, 0...."
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We...","[[0.002730799373239279, 0.0026100073009729385,...","[0.002730799373239279, 0.0026100073009729385, ..."


In [29]:
def label_to_vector(labels):
    init = [0]*28
    for label in labels:
        init[label] = 1
    return init

In [30]:
dataset['label_to_vector'] = dataset['labels'].apply(label_to_vector)
dataset

,text,labels,tokenized_text,emotion,flattened_padded_emotion,label_to_vector
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t...","[[0.003761823521926999, 0.0014409106224775314,...","[0.003761823521926999, 0.0014409106224775314, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi...","[[0.0026063367258757353, 0.0013388615334406495...","[0.0026063367258757353, 0.0013388615334406495,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]","[[0.0019205614225938916, 0.001291286083869636,...","[0.0019205614225938916, 0.001291286083869636, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]","[[0.0035849562846124172, 0.001992727629840374,...","[0.0035849562846124172, 0.001992727629840374, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]","[[0.0049132914282381535, 0.005827016197144985,...","[0.0049132914282381535, 0.005827016197144985, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ...","[[0.0035167797468602657, 0.0014414149336516857...","[0.0035167797468602657, 0.0014414149336516857,...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax...","[[0.002665032632648945, 0.002374983159825206, ...","[0.002665032632648945, 0.002374983159825206, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]","[[0.02350100688636303, 0.008980895392596722, 0...","[0.02350100688636303, 0.008980895392596722, 0....","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We...","[[0.002730799373239279, 0.0026100073009729385,...","[0.002730799373239279, 0.0026100073009729385, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."


In [31]:
dataset['index'] = list(range(len(dataset)))
dataset

,text,labels,tokenized_text,emotion,flattened_padded_emotion,label_to_vector,index
0,My favourite food is anything I didn't have to...,[27],"[My, favourite, food, is, anything, I, didn, t...","[[0.003761823521926999, 0.0014409106224775314,...","[0.003761823521926999, 0.0014409106224775314, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"Now if he does off himself, everyone will thin...",[27],"[Now, if, he, does, off, himself, everyone, wi...","[[0.0026063367258757353, 0.0013388615334406495...","[0.0026063367258757353, 0.0013388615334406495,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,WHY THE FUCK IS BAYLESS ISOING,[2],"[WHY, THE, FUCK, IS, BAYLESS, ISOING]","[[0.0019205614225938916, 0.001291286083869636,...","[0.0019205614225938916, 0.001291286083869636, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
3,To make her feel threatened,[14],"[To, make, her, feel, threatened]","[[0.0035849562846124172, 0.001992727629840374,...","[0.0035849562846124172, 0.001992727629840374, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",3
4,Dirty Southern Wankers,[3],"[Dirty, Southern, Wankers]","[[0.0049132914282381535, 0.005827016197144985,...","[0.0049132914282381535, 0.005827016197144985, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
...,...,...,...,...,...,...,...
1836,it’s happened before?! love my hometown of bea...,"[7, 18]","[it, s, happened, before, love, my, hometown, ...","[[0.0035167797468602657, 0.0014414149336516857...","[0.0035167797468602657, 0.0014414149336516857,...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",1836
1837,Talked with my parents. It was so relaxing and...,[15],"[Talked, with, my, parents, It, was, so, relax...","[[0.002665032632648945, 0.002374983159825206, ...","[0.002665032632648945, 0.002374983159825206, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1837
1838,"OMG, is this the one that wore white? 😳","[7, 26]","[OMG, is, this, the, one, that, wore, white, 😳]","[[0.02350100688636303, 0.008980895392596722, 0...","[0.02350100688636303, 0.008980895392596722, 0....","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",1838
1839,Nope! His real name starts with a G. Weird spe...,"[10, 11]","[Nope, His, real, name, starts, with, a, G, We...","[[0.002730799373239279, 0.0026100073009729385,...","[0.002730799373239279, 0.0026100073009729385, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...",1839


In [32]:
# train, validation, test split
indices = dataset['index'].to_list()
X1 = dataset['flattened_padded_emotion'].to_list()
y1 = dataset['label_to_vector'].to_list()
X1_train, X1_temp, y1_train, y1_temp, indices_train, indices_temp = train_test_split(X1, y1, indices, test_size=0.3, random_state=42)
X1_val, X1_test, y1_val, y1_test, indices_val, indices_test = train_test_split(X1_temp, y1_temp, indices_temp, test_size=0.5, random_state=42)

In [33]:
print("number of X1_train: ", len(X1_train), ", number of y1_train:", len(y1_train))
print("number of X1_val: ", len(X1_val), ", number of y1_val:", len(y1_val))
print("number of X1_test: ", len(X1_test), ", number of y1_test:", len(y1_test))

number of X1_train:  1288 , number of y1_train: 1288
number of X1_val:  276 , number of y1_val: 276
number of X1_test:  277 , number of y1_test: 277


In [34]:
# customizing dataset class

class Dataset1(Dataset):
    def __init__(self, X, y, indices):
        self.X_train = torch.tensor(X)
        self.y_train = torch.tensor(y)
        self.indices = indices
 
    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx], self.indices[idx]

In [35]:
class Model1(nn.Module):

    def __init__(self, input_size=1316, num_classes=28):
        super(Model1, self).__init__()
        self.linear1 = nn.Linear(input_size, input_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(input_size, num_classes) 
        #self.sigmoid_layers = nn.ModuleList([nn.Sigmoid() for _ in range(num_classes)])

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        out = self.linear2(x)
        #output = [sigmoid(x2) for sigmoid in self.sigmoid_layers] 
        #output = torch.stack(output, dim=1)

        return out, x

In [36]:
# create train, validation, test dataset

train_dataset1 = Dataset1(X1_train, y1_train, indices_train)
validation_dataset1 = Dataset1(X1_val, y1_val, indices_val)
test_dataset1 = Dataset1(X1_test, y1_test, indices_test)

# create train, validation, test dataloader
batch_size = 32
train_dataloader = DataLoader(train_dataset1, batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset1, batch_size=32)
test_dataloader = DataLoader(test_dataset1, batch_size)

In [37]:
input_size = len(dataset['flattened_padded_emotion'][0]))
model1 = Model1(input_size=input_size)
optimizer = optim.Adam(model1.parameters(), lr=1e-3)

In [38]:
def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

In [39]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()

    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)

    return {"auc_micro": auc_micro}

In [40]:
torch.manual_seed(42)

epochs = 100
patience = 5
best_val_loss = np.Inf
current_patience = 0
# word_embedding_dict = dict()

for epoch in range(epochs):
    model1.train()
    total_train_loss = 0.0

    for inputs, labels, indices in train_dataloader:
        optimizer.zero_grad()
        outputs, embedding = model1(inputs)
        train_loss = loss_fn(outputs, labels)
        train_loss.backward()
        optimizer.step()
        total_train_loss += train_loss.item()
        # word_embedding_dict[indices] = embedding

    model1.eval()
    total_val_loss = 0.0
    fin_labels = []
    fin_outputs = []

    with torch.inference_mode():
        for inputs, labels, indices in validation_dataloader:
            outputs, embedding = model1(inputs)
            val_loss = loss_fn(outputs, labels)
            total_val_loss += val_loss.item()
            # word_embedding_dict[indices] = embedding
            fin_labels.extend(labels)
            fin_outputs.extend(torch.sigmoid(outputs))

    avg_train_loss = total_train_loss / len(train_dataloader.dataset)
    avg_val_loss = total_val_loss / len(validation_dataloader.dataset)
    auc_score = log_metrics(fin_outputs, fin_labels)["auc_micro"]

    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, ACU Score: {auc_score}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        current_patience = 0
        torch.save(model1.state_dict(), 'best_model.pth')
        print("Model saved as current validation loss is: ", best_val_loss)
    else:
        current_patience += 1
    
    if current_patience >= patience:
        print(f'Early stopping at epoch {epoch+1} as there was no improvement in validation loss.')
        break

Epoch 1/100, Training Loss: 0.009655867813166625, Validation Loss: 0.005800380870915841, ACU Score: 0.7227630588256813
Model saved as current validation loss is:  0.005800380870915841
Epoch 2/100, Training Loss: 0.0053563219327363906, Validation Loss: 0.00528937831952952, ACU Score: 0.7726098447206863
Model saved as current validation loss is:  0.00528937831952952
Epoch 3/100, Training Loss: 0.0050797796351198824, Validation Loss: 0.0051360895884209785, ACU Score: 0.7995413864790551
Model saved as current validation loss is:  0.0051360895884209785
Epoch 4/100, Training Loss: 0.004857034393534157, Validation Loss: 0.004932128972765328, ACU Score: 0.8110189176172217
Model saved as current validation loss is:  0.004932128972765328
Epoch 5/100, Training Loss: 0.004576732676406825, Validation Loss: 0.004723669394202854, ACU Score: 0.8339442591768429
Model saved as current validation loss is:  0.004723669394202854
Epoch 6/100, Training Loss: 0.004278739648205894, Validation Loss: 0.004587555

In [41]:
torch.manual_seed(42)
word_embedding_dict = dict()
model1.eval
with torch.inference_mode():
    for inputs, _, indices in train_dataloader:
        _, embedding = model1(inputs)
        word_embedding_dict[indices] = embedding
    for inputs, _, indices in validation_dataloader:
        _, embedding = model1(inputs)
        word_embedding_dict[indices] = embedding
    for inputs, _, indices in test_dataloader:
        _, embedding = model1(inputs)
        word_embedding_dict[indices] = embedding

In [42]:
# word classes (final output)

flattened_word_list = [word for word_list in dataset['tokenized_text'] for word in word_list if emoji.emoji_count(word) == 0]
unique_word_list = sorted(list(set(flattened_word_list)))

word_to_idx = {word: idx for idx, word in enumerate(unique_word_list)}
idx_to_word = {idx: word for idx, word in enumerate(unique_word_list)}

print('the number of words: ', len(idx_to_word))
print("idx_to_word: ", idx_to_word)
print("word_to_idx: ", word_to_idx)

the number of words:  4503
idx_to_word:  {0: 'A', 1: 'AA', 2: 'AB', 3: 'ABH', 4: 'AHAHA', 5: 'AM', 6: 'AMA', 7: 'AND', 8: 'ANGEL', 9: 'ANYONE', 10: 'APPROVE', 11: 'AR', 12: 'ARE', 13: 'AROUND', 14: 'AS', 15: 'ASAP', 16: 'ASD', 17: 'ATF', 18: 'ATM', 19: 'Aaahahahha', 20: 'Ab', 21: 'Absolutely', 22: 'Actually', 23: 'Acura', 24: 'Addison', 25: 'After', 26: 'Again', 27: 'Aggieopoly', 28: 'Aghulas', 29: 'Agreed', 30: 'Ah', 31: 'Ahem', 32: 'Ahh', 33: 'Ahhh', 34: 'Ahhhh', 35: 'AiG', 36: 'Al', 37: 'Alarming', 38: 'Alert', 39: 'All', 40: 'Allow', 41: 'Alright', 42: 'Also', 43: 'Although', 44: 'Alvalade', 45: 'Always', 46: 'Am', 47: 'Amateurs', 48: 'Amazing', 49: 'Amazon', 50: 'Ameen', 51: 'America', 52: 'American', 53: 'And', 54: 'Angel', 55: 'Animal', 56: 'Animation', 57: 'Another', 58: 'Any', 59: 'Anymore', 60: 'Anything', 61: 'Anytime', 62: 'Anyway', 63: 'Apologies', 64: 'Appalachia', 65: 'Apple', 66: 'Applying', 67: 'Are', 68: 'Aren', 69: 'Argh', 70: 'Argue', 71: 'Arizona', 72: 'Arm', 73: '

In [43]:
word_embedding_dict

{tensor([1389,  315, 1483,   96, 1183,  296,  573,  152,  699, 1186,  454,  983,
         1090, 1213,   94,  914,  205,  814,  304, 1037,   13, 1017,  229,  698,
          171, 1769,  522,  885, 1777,   14,  856,  151]): tensor([[0.1158, 0.2528, 0.0378,  ..., 0.2255, 0.1758, 0.0000],
         [0.0886, 0.1335, 0.1508,  ..., 0.1249, 0.0891, 0.0000],
         [0.2391, 0.0000, 0.3805,  ..., 0.1559, 0.1945, 0.0000],
         ...,
         [0.0753, 0.1029, 0.0438,  ..., 0.1978, 0.0529, 0.0000],
         [0.1144, 0.1958, 0.1020,  ..., 0.1424, 0.0959, 0.0000],
         [0.1308, 0.0431, 0.1764,  ..., 0.0362, 0.1899, 0.0000]]),
 tensor([1735,   85, 1538, 1117, 1373, 1288,  345,  114,  750,   36, 1531,  193,
          646, 1530,  579,  132, 1316,  340, 1134,  993, 1433, 1522, 1216,  683,
          878, 1391,  409,  858, 1695, 1499,  969, 1644]): tensor([[0.2569, 0.1662, 0.0722,  ..., 0.1924, 0.3354, 0.0000],
         [0.0827, 0.0987, 0.1177,  ..., 0.0887, 0.0175, 0.0000],
         [0.1618, 0.1571

In [44]:
len(word_embedding_dict)

59

In [45]:
# 모델2 데이터 페어 구성
model2_dataset = dict()
for i in range(len(word_embedding_dict)):
    key = list(word_embedding_dict.keys())[i]
    value = word_embedding_dict[key]
    for j in range(len(key)):
        data_idx = key[j].item()
        words = list(dataset[dataset['index'] == data_idx]['tokenized_text'])[0]
        embeddings = value[j]
        for k in range(len(words)):
            word = words[k]
            if emoji.emoji_count(word) == 0:
                embedding = embeddings[k*28:(k+1)*28]
                word_idx = word_to_idx[word]
                model2_dataset[word_idx] = embedding 

In [46]:
model2_dataset

{440: tensor([0.0000, 0.0000, 0.1642, 0.1316, 0.1667, 0.0903, 0.1326, 0.1784, 0.1796,
         0.1138, 0.0957, 0.1733, 0.1667, 0.0896, 0.0958, 0.2662, 0.0000, 0.1755,
         0.0817, 0.1010, 0.0000, 0.1683, 0.2049, 0.0000, 0.1333, 0.1344, 0.1394,
         0.1411]),
 4099: tensor([0.0000, 0.2034, 0.1185, 0.0000, 0.1733, 0.2139, 0.1182, 0.1245, 0.0691,
         0.0916, 0.0439, 0.1237, 0.0966, 0.1697, 0.0866, 0.2224, 0.0670, 0.0970,
         0.1348, 0.1937, 0.0824, 0.1234, 0.1816, 0.1389, 0.0000, 0.1316, 0.0683,
         0.1195]),
 2692: tensor([0.0000, 0.0000, 0.2732, 0.3842, 0.1081, 0.0838, 0.0000, 0.0000, 0.4043,
         0.3638, 0.1368, 0.2273, 0.0000, 0.2130, 0.1201, 0.3150, 0.2690, 0.0000,
         0.5215, 0.1329, 0.0000, 0.0932, 0.0000, 0.0000, 0.1842, 0.0000, 0.2663,
         0.0000]),
 4340: tensor([0.1701, 0.0646, 0.1650, 0.0000, 0.3537, 0.1411, 0.2893, 0.0000, 0.0000,
         0.0000, 0.2170, 0.3072, 0.1426, 0.3665, 0.0000, 0.1701, 0.3447, 0.0000,
         0.1246, 0.2363, 0.02

In [47]:
X2 = list(model2_dataset.values())
X2 = [embedding.tolist() for embedding in X2]
y2 = list(model2_dataset.keys())

X2_train, X2_temp, y2_train, y2_temp = train_test_split(X2, y2, test_size=0.1, random_state=42)
X2_val, X2_test, y2_val, y2_test = train_test_split(X2_temp, y2_temp, test_size=0.5, random_state=42)

print("X2_train data size: ", len(X2_train))
print("X2_val data size: ", len(X2_val))
print("X2_test data size: ", len(X2_test))

X2_train data size:  4052
X2_val data size:  225
X2_test data size:  226


In [48]:
class Dataset2(Dataset):
    def __init__(self, X, y):
        self.X_train = torch.tensor(X)
        self.y_train = torch.tensor(y)

    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx]

In [49]:
class Model2(nn.Module):

    def __init__(self, input_size=28, num_classes=4503):
        super(Model2, self).__init__()
        self.linear1 = nn.Linear(input_size, num_classes)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(num_classes, num_classes)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        out = self.linear2(x)

        return out

In [50]:
train_dataset2 = Dataset2(X2_train, y2_train)
validation_dataset2 = Dataset2(X2_val, y2_val)
test_dataset2 = Dataset2(X2_test, y2_test)

batch_size=32
train_dataloader2 = DataLoader(train_dataset2, batch_size, shuffle=True)
validation_dataloader2 = DataLoader(validation_dataset2, batch_size=32)
test_dataloader2 = DataLoader(test_dataset2, batch_size=32)

In [51]:
num_classes = len(idx_to_word)
model2 = Model2(num_classes=num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=1e-3)

In [52]:
torch.manual_seed(42)

epochs = 100
patience = 5
best_val_loss = np.Inf
current_patience = 0

for epoch in range(epochs):
    model2.train()
    # total_train_acc = 0.0
    # total_train_count = 0.0
    total_train_loss = 0.0

    for inputs, labels in train_dataloader2:
        optimizer.zero_grad()
        outputs = model2(inputs)
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()
        
        # total_train_acc += (outputs.argmax(1) == labels).sum().item()
        # total_train_count += labels.size(0)
        total_train_loss += train_loss.item()
        
    model2.eval()
    # total_val_acc = 0.0
    # total_val_count = 0.0
    total_val_loss = 0.0

    with torch.inference_mode():
        for inputs, labels in validation_dataloader2:
            outputs = model2(inputs)
            val_loss = criterion(outputs, labels)
            # total_val_acc += (outputs.argmax(1) == labels).sum().item()
            # total_val_count += labels.size(0)
            total_val_loss += val_loss.item()

    # train_accuracy = total_train_acc / total_train_count
    # val_accuracy = total_val_acc / total_val_count
    avg_train_loss = total_train_loss / len(train_dataloader2.dataset)
    avg_val_loss = total_val_loss / len(validation_dataloader2.dataset)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        current_patience = 0
        torch.save(model2.state_dict(), 'best_model2.pth')
        print("Model saved as current validation loss is ", best_val_loss)

    else:
        current_patience += 1

    if current_patience >= patience:
        print(f'Early stopping at epoch {epoch+1} as there was no improvement in validation loss.')
        break



Epoch 1/100, Train Loss: 0.27134026332527605, Val Loss: 0.30211034562852646
Model saved as current validation loss is  0.30211034562852646
Epoch 2/100, Train Loss: 0.2372627717250662, Val Loss: 0.3431283484564887
Epoch 3/100, Train Loss: 0.13885037914315662, Val Loss: 0.4572447035047743
Epoch 4/100, Train Loss: 0.061108798553536466, Val Loss: 0.5596921412150065
Epoch 5/100, Train Loss: 0.034506489190862306, Val Loss: 0.613073844909668
Epoch 6/100, Train Loss: 0.026751960628654033, Val Loss: 0.6347761705186632
Early stopping at epoch 6 as there was no improvement in validation loss.


In [53]:
#모델 1 테스트
torch.manual_seed(42)

model1.load_state_dict(torch.load('best_model.pth'))
model1.eval()
total_test_loss_model1 = 0.0
fin_labels_model1 = []
fin_outputs_model1 = []

with torch.inference_mode():
    for inputs, labels, _ in test_dataloader:
        outputs, _ = model1(inputs)
        test_loss = loss_fn(outputs, labels)
        total_test_loss_model1 += test_loss.item()
        fin_labels_model1.extend(labels)
        fin_outputs_model1.extend(torch.sigmoid(outputs))

avg_test_loss_model1 = total_test_loss_model1 / len(test_dataloader.dataset)
auc_score_model1 = log_metrics(fin_outputs_model1, fin_labels_model1)["auc_micro"]
print(f"Model 1 Test Loss: {avg_test_loss_model1}, AUC Score: {auc_score_model1}")

model2.load_state_dict(torch.load('best_model2.pth'))
model2.eval()
total_test_loss_model2 = 0.0
fin_labels_model2 = []
fin_outputs_model2 = []

with torch.inference_mode():
    for inputs, labels in test_dataloader2:
        outputs = model2(inputs)
        test_loss = criterion(outputs, labels)
        total_test_loss_model2 += test_loss.item()

avg_test_loss_model2 = total_test_loss_model2 / len(test_dataloader2.dataset)
print(f"Model 2 Test Loss: {avg_test_loss_model2}")

Model 1 Test Loss: 0.004193168657996594, AUC Score: 0.86710635058186
Model 2 Test Loss: 0.3007899216846027


In [54]:
def Emoji_to_Word_Translator(emoji_str):
    emoji_name = emoji.demojize(emoji_str)[1:-1]
    emoji_emotion = word_emotion(emoji_name)
    input = torch.tensor([emoji_emotion + [0]*(1316-len(emoji_emotion))])
    
    model1.eval()
    model2.eval()

    with torch.inference_mode():

        _, embedding = model1(input)
        embedding = embedding[:, :28]

        outputs = model2(embedding)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

    k = 1
    _, top_indices = torch.topk(probabilities, k)
    translated_words = ""
    for idx in top_indices[0]:
        translated_words += idx_to_word[idx.item()]
        translated_words += " "

    return translated_words
    

In [55]:
Emoji_to_Word_Translator('💕')

'Their '

In [56]:
# 평가 데이터셋
eval_dataset = load_dataset('go_emotions', 'simplified')
eval_dataset = eval_dataset['validation'].to_pandas()

Using the latest cached version of the dataset since go_emotions couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'simplified' at C:\Users\김성연\.cache\huggingface\datasets\go_emotions\simplified\0.0.0\add492243ff905527e67aeb8b80c082af02207c3 (last modified on Thu Jan 11 19:31:53 2024).


In [57]:
cols = ['text', 'labels']
new_eval_dataset = pd.DataFrame(columns=cols)
k=0

for i in range(len(eval_dataset)):
    text = eval_dataset.iloc[i]['text']
    if emoji.emoji_count(text) != 0:
        new_eval_dataset.loc[k] = eval_dataset.iloc[i][['text', 'labels']]
        k += 1
        
new_eval_dataset

,text,labels
0,[NAME] is such a legendary daddy 😩,[27]
1,"Hahahah thank you so much, username does not c...","[1, 10, 15]"
2,Any response helps and makes me feel like I’m ...,[15]
3,Man I love Canada🇨🇦,[18]
4,[NAME]. Get off of the computer my man. Get so...,"[1, 5]"
...,...,...
109,Disgusting. Need to get the tooth hole fixed😂,[11]
110,Damn. Is that what those shorts are supposed t...,"[3, 7]"
111,Love my 90DF Family Reddit too! 💙,[18]
112,Very very nice to see this irl. Nice post 👌🏼,"[0, 4]"


In [58]:
def emoji_translator_to_text(text):
    for char in text:
        if emoji.emoji_count(char) == 1:
            text = text.replace(char, Emoji_to_Word_Translator(char))
    return text

In [59]:
def emoji_delete(text):
    for char in text:
        if emoji.emoji_count(char) == 1:
            text = text.replace(char, '')
    return text

In [60]:
def emoji_extractor(text):
    only_emoji = ""
    for char in text:
        if emoji.emoji_count(char) == 1:
            only_emoji += char
            only_emoji += " "
    return only_emoji

In [61]:
emoji_extractor(new_eval_dataset['text'][113])

'🤔 '

In [62]:
new_eval_dataset['emoji_delete_text'] = new_eval_dataset['text'].apply(emoji_delete)
new_eval_dataset['emoji_translate_text'] = new_eval_dataset['text'].apply(emoji_translator_to_text)
new_eval_dataset['label_to_vector'] = new_eval_dataset['labels'].apply(label_to_vector)
new_eval_dataset['emoji_delete_text_emotion'] = new_eval_dataset['emoji_delete_text'].apply(word_emotion)
new_eval_dataset['emoji_translate_text_emotion'] = new_eval_dataset['emoji_translate_text'].apply(word_emotion)
new_eval_dataset['only_emoji'] = new_eval_dataset['text'].apply(emoji_extractor)
new_eval_dataset['only_emoji'] = new_eval_dataset['only_emoji'].apply(Emoji_to_Word_Translator)
new_eval_dataset['only_emoji_emotion'] = new_eval_dataset['only_emoji'].apply(word_emotion)
new_eval_dataset

KeyboardInterrupt: 

In [ ]:
# 바다에 색소 한 방울 떨어트리는 실험 ^^

evaluation = torch.nn.MSELoss()

# emoji translator 사용하지 않았을 때 Loss evaluation
raw_loss = evaluation(torch.tensor(new_eval_dataset['emoji_delete_text_emotion'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
#raw_loss_round = round(raw_loss.item(), 5)
print(f"emoji translator 사용하지 않았을 때 Loss: {raw_loss}")
# 0.0319

# emoji translator 사용했을 때 Loss
trans_loss = evaluation(torch.tensor(new_eval_dataset['emoji_translate_text_emotion'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
#trans_loss_round = round(trans_loss.item(), 4)
print(f"emoji translator 사용했을 때 Loss: {trans_loss}")
# 0.0315

emoji translator 사용하지 않았을 때 Loss: 0.031954389065504074
emoji translator 사용했을 때 Loss: 0.03158906474709511


In [ ]:
# emoji weighted emotion 열 추가
emoji_weight_1 = []
emoji_weight_3 = []
emoji_weight_4 = []
emoji_weight_5 = []
emoji_weight_6 = []
emoji_weight_7 = []
emoji_weight_8 = []
emoji_weight_9 = []
emoji_weight_10 = []
emoji_weight_20 = []
emoji_weight_30 = []
emoji_weight_50 = []
emoji_weight_70 = []

for i in range(len(new_eval_dataset)):
    text_emotion = torch.tensor(new_eval_dataset.iloc[i]['emoji_delete_text_emotion'])
    emoji_emotion = torch.tensor(new_eval_dataset.iloc[i]['only_emoji_emotion'])
    w_1_sum = emoji_emotion * 0.01 + text_emotion * 0.99
    w_3_sum = emoji_emotion * 0.03 + text_emotion * 0.97
    w_4_sum = emoji_emotion * 0.04 + text_emotion * 0.96
    w_5_sum = emoji_emotion * 0.05 + text_emotion * 0.95
    w_6_sum = emoji_emotion * 0.06 + text_emotion * 0.94
    w_7_sum = emoji_emotion * 0.07 + text_emotion * 0.93
    w_8_sum = emoji_emotion * 0.08 + text_emotion * 0.92
    w_9_sum = emoji_emotion * 0.09 + text_emotion * 0.91
    w_10_sum = emoji_emotion * 0.1 + text_emotion * 0.9
    w_20_sum = emoji_emotion * 0.2 + text_emotion * 0.8
    w_30_sum = emoji_emotion * 0.3 + text_emotion * 0.7
    w_50_sum = emoji_emotion * 0.5 + text_emotion * 0.5
    w_70_sum = emoji_emotion * 0.7 + text_emotion * 0.3
    emoji_weight_1.append(w_1_sum.tolist())
    emoji_weight_3.append(w_3_sum.tolist())
    emoji_weight_4.append(w_4_sum.tolist())
    emoji_weight_5.append(w_5_sum.tolist())
    emoji_weight_6.append(w_6_sum.tolist())
    emoji_weight_7.append(w_7_sum.tolist())
    emoji_weight_8.append(w_8_sum.tolist())
    emoji_weight_9.append(w_9_sum.tolist())
    emoji_weight_10.append(w_10_sum.tolist())
    emoji_weight_20.append(w_20_sum.tolist())
    emoji_weight_30.append(w_30_sum.tolist())
    emoji_weight_50.append(w_50_sum.tolist())
    emoji_weight_70.append(w_70_sum.tolist())

new_eval_dataset["emoji_weight_1"] = emoji_weight_1
new_eval_dataset["emoji_weight_3"] = emoji_weight_3
new_eval_dataset["emoji_weight_4"] = emoji_weight_4
new_eval_dataset["emoji_weight_5"] = emoji_weight_5
new_eval_dataset["emoji_weight_6"] = emoji_weight_6
new_eval_dataset["emoji_weight_7"] = emoji_weight_7
new_eval_dataset["emoji_weight_8"] = emoji_weight_8
new_eval_dataset["emoji_weight_9"] = emoji_weight_9
new_eval_dataset["emoji_weight_10"] = emoji_weight_10
new_eval_dataset["emoji_weight_20"] = emoji_weight_20
new_eval_dataset["emoji_weight_30"] = emoji_weight_30
new_eval_dataset["emoji_weight_50"] = emoji_weight_50
new_eval_dataset["emoji_weight_70"] = emoji_weight_70


In [ ]:
new_eval_dataset

,text,labels,emoji_delete_text,emoji_translate_text,label_to_vector,emoji_delete_text_emotion,emoji_translate_text_emotion,only_emoji,only_emoji_emotion,emoji_weight_1,...,emoji_weight_5,emoji_weight_6,emoji_weight_7,emoji_weight_8,emoji_weight_9,emoji_weight_10,emoji_weight_20,emoji_weight_30,emoji_weight_50,emoji_weight_70
0,[NAME] is such a legendary daddy 😩,[27],[NAME] is such a legendary daddy,[NAME] is such a legendary daddy Their,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.8371853232383728, 0.0010130126029253006, 0....","[0.7986921072006226, 0.0009594232542440295, 0....",Their,"[0.0034218698274344206, 0.0018512787064537406,...","[0.8288477063179016, 0.0010213953210040927, 0....",...,"[0.7954971194267273, 0.0010549259604886174, 0....","[0.7871595621109009, 0.0010633085621520877, 0....","[0.7788218855857849, 0.001071691163815558, 0.0...","[0.7704842686653137, 0.00108007388189435, 0.00...","[0.7621466517448425, 0.0010884565999731421, 0....","[0.7538089752197266, 0.0010968392016366124, 0....","[0.6704326272010803, 0.0011806658003479242, 0....","[0.5870562791824341, 0.0012644925154745579, 0....","[0.4203035831451416, 0.0014321457128971815, 0....","[0.2535509169101715, 0.0015997989103198051, 0...."
1,"Hahahah thank you so much, username does not c...","[1, 10, 15]","Hahahah thank you so much, username does not c...","Hahahah thank you so much, username does not c...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.007812465541064739, 0.5283613801002502, 0.0...","[0.00707726925611496, 0.5504684448242188, 0.00...",Their,"[0.0034218698274344206, 0.0018512787064537406,...","[0.007768559735268354, 0.5230963230133057, 0.0...",...,"[0.0075929355807602406, 0.5020358562469482, 0....","[0.007549029774963856, 0.4967707693576813, 0.0...","[0.007505123969167471, 0.4915056824684143, 0.0...","[0.007461218163371086, 0.48624056577682495, 0....","[0.007417312357574701, 0.48097550868988037, 0....","[0.007373405620455742, 0.47571036219596863, 0....","[0.006934346631169319, 0.4230593740940094, 0.0...","[0.006495286710560322, 0.3704083561897278, 0.0...","[0.005617167800664902, 0.26510632038116455, 0....","[0.004739048890769482, 0.1598043143749237, 0.0..."
2,Any response helps and makes me feel like I’m ...,[15],Any response helps and makes me feel like I’m ...,Any response helps and makes me feel like I’m ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.008541022427380085, 0.0018819638062268496, ...","[0.011030198074877262, 0.0018687736010178924, ...",Their,"[0.0034218698274344206, 0.0018512787064537406,...","[0.00848983135074377, 0.0018816570518538356, 0...",...,"[0.00828506425023079, 0.0018804295687004924, 0...","[0.008233873173594475, 0.0018801226979121566, ...","[0.008182681165635586, 0.0018798158271238208, ...","[0.008131490088999271, 0.0018795090727508068, ...","[0.008080299012362957, 0.001879202201962471, 0...","[0.008029107004404068, 0.0018788952147588134, ...","[0.007517192047089338, 0.0018758268561214209, ...","[0.007005276624113321, 0.0018727582646533847, ...","[0.005981446243822575, 0.0018666211981326342, ...","[0.004957615397870541, 0.0018604842480272055, ..."
3,Man I love Canada🇨🇦,[18],Man I love Canada🇨🇦,Man I love Canada🇨🇦,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0395917110145092, 0.0032190720085054636, 0....","[0.0395917110145092, 0.0032190720085054636, 0....",Their,"[0.0034218698274344206, 0.0018512787064537406,...","[0.03923001512885094, 0.0032053941395133734, 0...",...,"[0.037783220410346985, 0.003150682197883725, 0...","[0.03742152079939842, 0.003137004328891635, 0....","[0.03705982491374016, 0.0031233264598995447, 0...","[0.036698125302791595, 0.003109648823738098, 0...","[0.03633642941713333, 0.0030959707219153643, 0...","[0.03597472608089447, 0.0030822926200926304, 0...","[0.03235774114727974, 0.002945513231679797, 0....","[0.028740758076310158, 0.002808733843266964, 0...","[0.021506790071725845, 0.002535175299271941, 0...","[0.014272822067141533, 0.0022616167552769184, ..."
4,[

In [ ]:
# 교수님께서 이야기해주신 대로!! Weight 부여해서 

w_1_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_1'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_3_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_3'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_4_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_4'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_5_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_5'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_6_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_6'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_7_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_7'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_8_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_8'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_9_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_9'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_10_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_10'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_20_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_20'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_30_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_30'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_50_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_50'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
w_70_loss = evaluation(torch.tensor(new_eval_dataset['emoji_weight_70'].to_list()), torch.tensor(new_eval_dataset['label_to_vector'].to_list()))
print(f"emoji weight 1% emotion Loss: {w_1_loss}")
print(f"emoji weight 3% emotion Loss: {w_3_loss}")
print(f"emoji weight 4% emotion Loss: {w_4_loss}")
print(f"emoji weight 5% emotion Loss: {w_5_loss}")
print(f"emoji weight 6% emotion Loss: {w_6_loss}")
print(f"emoji weight 7% emotion Loss: {w_7_loss}")
print(f"emoji weight 8% emotion Loss: {w_8_loss}")
print(f"emoji weight 9% emotion Loss: {w_9_loss}")
print(f"emoji weight 10% emotion Loss: {w_10_loss}")
print(f"emoji weight 20% emotion Loss: {w_20_loss}")
print(f"emoji weight 30% emotion Loss: {w_30_loss}")
print(f"emoji weight 50% emotion Loss: {w_50_loss}")
print(f"emoji weight 70% emotion Loss: {w_70_loss}")

emoji weight 1% emotion Loss: 0.03191280737519264
emoji weight 3% emotion Loss: 0.03185516968369484
emoji weight 4% emotion Loss: 0.03183911368250847
emoji weight 5% emotion Loss: 0.03183156996965408
emoji weight 6% emotion Loss: 0.03183254599571228
emoji weight 7% emotion Loss: 0.03184202313423157
emoji weight 8% emotion Loss: 0.03186001256108284
emoji weight 9% emotion Loss: 0.031886518001556396
emoji weight 10% emotion Loss: 0.03192152455449104
emoji weight 20% emotion Loss: 0.032739754766225815
emoji weight 30% emotion Loss: 0.03440907597541809
emoji weight 50% emotion Loss: 0.04030097648501396
emoji weight 70% emotion Loss: 0.04959724098443985


In [ ]:
e2v = gsm.Word2Vec('emoji2vec.bin', binary=True)

TypeError: Word2Vec.__init__() got an unexpected keyword argument 'binary'

In [ ]:
vec